In [12]:
import requests
import pandas as pd
import tqdm

In [2]:
df = pd.read_excel("data/SA2_2021_AUST.xlsx")

In [13]:
df.head()

,SA2_CODE_2021,SA2_NAME_2021,CHANGE_FLAG_2021,CHANGE_LABEL_2021,SA3_CODE_2021,SA3_NAME_2021,SA4_CODE_2021,SA4_NAME_2021,GCCSA_CODE_2021,GCCSA_NAME_2021,STATE_CODE_2021,STATE_NAME_2021,AUS_CODE_2021,AUS_NAME_2021,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...


- SA2_NAME_2021: suburb name
- SA3_NAME_2021: council name
- SA4_NAME_2021: city name
- GCCSA_NAME_2021: region name
- STATE_NAME_2021: state name

# 1. Clean list of suburbs

Things to consider:

| Consideration                              | Raw example           | Clean example                 |
|--------------------------------------------|-----------------------|-------------------------------|
| Split names with '-' into separate suburbs | Emerald - Cockatoo    | [..., Emerald, Cockatoo, ...] |
| Remove any brackets                        | Blacktown (East)      | [..., Blacktown, ...]         |
| Remove cardinal directions                 | Pakenham - North East | [..., Pakenham, ...]          |
| Apply lowercase for website request        | Oakleigh              | [..., oakleigh, ...]          |
| Replace spaces in names with '+'           | Koo Wee Rup           | [..., koo+wee+rup, ...]       |

In [14]:
df[["SA2_NAME_2021", "SA3_NAME_2021", "SA4_NAME_2021", "GCCSA_NAME_2021", "STATE_NAME_2021"]]

,SA2_NAME_2021,SA3_NAME_2021,SA4_NAME_2021,GCCSA_NAME_2021,STATE_NAME_2021
0,Braidwood,Queanbeyan,Capital Region,Rest of NSW,New South Wales
1,Karabar,Queanbeyan,Capital Region,Rest of NSW,New South Wales
2,Queanbeyan,Queanbeyan,Capital Region,Rest of NSW,New South Wales
3,Queanbeyan - East,Queanbeyan,Capital Region,Rest of NSW,New South Wales
4,Queanbeyan West - Jerrabomberra,Queanbeyan,Capital Region,Rest of NSW,New South Wales
...,...,...,...,...,...
2468,Jervis Bay,Jervis Bay,Other Territories,Other Territories,Other Territories
2469,Norfolk Island,Norfolk Island,Other Territories,Other Territories,Other Territories
2470,Migratory - Offshore - Shipping (OT),Migratory - Offshore - Shipping (OT),Migratory - Offshore - Shipping (OT),Migratory - Offshore - Shipping (OT),Other Territories
2471,No usual address (OT),No usual address (OT),No usual address (OT),No usual address (OT),Other Territories


In [30]:
cardinal_directions = [
    "Central",
    "North",
    "North East",
    "East",
    "South East",
    "South",
    "South West",
    "West",
    "North West"
]

suburbs = []

for x in df["SA2_NAME_2021"]:
    if " - " in x:
        x = x.split(" - ")

    if "(" in x:
        x = x.split("(")
    
    if isinstance(x, list):
        for sub in x:
            suburbs.append(sub)
    else:
        suburbs.append(x)

In [33]:
[x for x in suburbs if "Sydney" in x]

['Sydney Airport',
 'Sydney (North)',
 'Sydney (South)',
 'North Sydney',
 'Sydney Olympic Park']

In [31]:
[x for x in suburbs if ")" in x]

['NSW)',
 'Rutherford (North)',
 'Rutherford (South)',
 'NSW)',
 'Baulkham Hills (West)',
 'Blacktown (East)',
 'Blacktown (North)',
 'Schofields (West)',
 'NSW)',
 'Sydney (North)',
 'Sydney (South)',
 'NSW)',
 'Panania (North)',
 'Panania (South)',
 'Kingsgrove (South)',
 'NSW)',
 'Canterbury (North)',
 'NSW)',
 'Chatswood (West)',
 'Wahroonga (West)',
 'Wahroonga (East)',
 'Warriewood (North)',
 'Warriewood (South)',
 'Dee Why (South)',
 'Greystanes (North)',
 'Epping (East)',
 'Epping (NSW)',
 'Shipping (NSW)',
 'NSW)',
 'Vic.)',
 'Vic.)',
 'Vic.)',
 'Vic.)',
 'Vic.)',
 'Essendon (West)',
 'Vic.)',
 'Southbank (West)',
 'Richmond (South)',
 'Vic.)',
 'Surrey Hills (West)',
 'Vic.)',
 'Surrey Hills (East)',
 'Vic.)',
 'Highett (West)',
 'Highett (East)',
 'Epping (Vic.)',
 'Roxburgh Park (South)',
 'Ferntree Gully (South)',
 'Vic.)',
 'Clayton (North)',
 'Vic.)',
 'Tarneit (West)',
 'Vic.)',
 'Vic.)',
 'Vic.)',
 'Shipping (Vic.)',
 'Vic.)',
 'Qld)',
 'Qld)',
 'Qld)',
 'Qld)',
 'Qld)

In [9]:
df

,SA2_CODE_2021,SA2_NAME_2021,CHANGE_FLAG_2021,CHANGE_LABEL_2021,SA3_CODE_2021,SA3_NAME_2021,SA4_CODE_2021,SA4_NAME_2021,GCCSA_CODE_2021,GCCSA_NAME_2021,STATE_CODE_2021,STATE_NAME_2021,AUS_CODE_2021,AUS_NAME_2021,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,901031003,Jervis Bay,0,No change,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,67.2296,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2469,901041004,Norfolk Island,0,No change,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,38.6510,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2470,997979799,Migratory - Offshore - Shipping (OT),0,No change,99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2471,999999499,No usual address (OT),0,No change,99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...


In [ ]:
def scrape_jina_ai(url):
    response = requests